## Fetching Point of Interest information from latitude, longitude data

airbnb data had latitude and longitudes of each of the listings. This two features can be used to get the information about surrounding POI's.This is important beacause listings near tourist spots or cafes and restaurants has generally high booking rates. Thus count of these POI's can be obtained in given radius range arround the coordinates in order to determine the attractiveness of the location of the listing. 

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
l=pd.read_csv("train_latLong.csv")


In [2]:
df=l.head(20)
df

,id,latitude,longitude
0,1,25.98676,-80.12252
1,2,40.76198,-73.98718
2,3,34.00697,-118.45050
3,4,45.18900,-93.13541
4,5,33.76120,-118.13839
5,6,40.81575,-73.94913
6,7,40.70423,-73.81440
7,8,40.73624,-74.07250
8,9,41.48188,-71.42375
9,10,34.06883,-118.13334


df is subset of larger aibnb data, with only two features of latitude and longitude from it. Goal is to create new features with count of tourist attraction, bars,cafes,pubs and restaurants. I have done it using two methods.
First is free but time consuming ,using overpass api (https://wiki.openstreetmap.org/wiki/Overpass_API)

second is faster but could be very expensive if api requests goes past free limits using Google api .(https://developers.google.com/places/web-service/search)

## Finding attractiveness of location using overpass api

Overpass api can be used to fetch information about wide range of POI's. I have used 5 which I though could affect the decision of airbnb customer while booking the lisiting. Other POI's which can be used similarly can be found here https://wiki.openstreetmap.org/wiki/Map_Features.

Code will find tourist attractions, bars, cafes, pubs and restaurants in 500 meter radius around the coordinates. Radius can be changed by changing the parameter of around in the code below.

In [7]:
p1=[]
p15=[]
p16=[]
p17=[]
p18=[]

count=1
for index, row in df.iterrows():
    
    latog=row.latitude
    longog=row.longitude
    #print(latog)
    #print(lat,long)
    
    overpass_url = "http://overpass-api.de/api/interpreter"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
# heades are required to unblock api request from above web browsers
    overpass_query = f"""
    [out:json];
    (
    node(around:500.0,{latog},{longog})["tourism"="attraction"];
    way(around:500.0,{latog},{longog})["tourism"="attraction"];
    relation(around:500.0,{latog},{longog})["tourism"="attraction"];

    node(around:500.0,{latog},{longog})["amenity"="bar"];
    way(around:500.0,{latog},{longog})["amenity"="bar"];
    relation(around:500.0,{latog},{longog})["amenity"="bar"];

    node(around:500.0,{latog},{longog})["amenity"="cafe"];
    way(around:500.0,{latog},{longog})["amenity"="cafe"];
    relation(around:500.0,{latog},{longog})["amenity"="cafe"];
    

    node(around:500.0,{latog},{longog})["amenity"="pub"];
    way(around:500.0,{latog},{longog})["amenity"="pub"];
    relation(around:500.0,{latog},{longog})["amenity"="pub"];

    node(around:500.0,{latog},{longog})["amenity"="restaurant"];
    way(around:500.0,{latog},{longog})["amenity"="restaurant"];
    relation(around:500.0,{latog},{longog})["amenity"="restaurant"];
    
    
        

    
    
    );
    out center;
    """ 
    
    try:
        response = requests.get(overpass_url, 
                        params={'data': overpass_query},headers=headers)
        data = response.json()
        #print(data)
        print(count)# count is used to keep track of progess
        for el in data['elements']:
            print(el['tags'])
            print("\n")
            break
        #print("\n")
    except json.decoder.JSONDecodeError:
        p1.append('NA')
        p15.append('NA')
        p16.append('NA')
        p17.append('NA')
        p18.append('NA')

        
        count=count+1
        continue
    except ConnectionError as e:
        p1.append('NA')
        p15.append('NA')
        p16.append('NA')
        p17.append('NA')
        p18.append('NA')

        
        count=count+1
        continue
    
    p1c=[]
    p15c=[]
    p16c=[]
    p17c=[]
    p18c=[]
    

    for element in data['elements']:
        if (element['type'] == 'node' and 'tourism' in element['tags']) or ('center' in element and 'tourism' in element['tags']) :
            if (element['tags']['tourism']=='attraction'):
                 for el in element:
                        if el=='id':
                            p1c.append(id)
                                           

                
        elif (element['type'] == 'node' and 'amenity' in element['tags']) or ('center' in element and 'amenity' in element['tags']):
            if (element['tags']['amenity']=='bar'):
                for el in element:
                        if el=='id':
                            p15c.append(id)
                
            elif (element['tags']['amenity']=='cafe'):
                for el in element:
                        if el=='id':
                            p16c.append(id)
                
        
            elif (element['tags']['amenity']=='pub'):
                for el in element:
                        if el=='id':
                            p17c.append(id)
        
            elif (element['tags']['amenity']=='restaurant'):
                for el in element:
                        if el=='id':
                            p18c.append(id)
                

                            
      
    p1cd=np.array(p1c)
    p15cd=np.array(p15c)
    p16cd=np.array(p16c)
    p17cd=np.array(p17c)
    p18cd=np.array(p18c)

        
    p1.append(len(p1cd))
    p15.append(len(p15cd))
    p16.append(len(p16cd))
    p17.append(len(p17cd))
    p18.append(len(p18cd))
    
    count=count+1
    


1
{'addr:housenumber': '4111', 'addr:postcode': '33019', 'addr:street': 'South Ocean Drive', 'amenity': 'restaurant', 'name': 'Terrazas at Hyde Hollywood', 'opening_hours': 'Su-Th 07:00-22:00; Fr-Sa 07:00-23:00', 'phone': '+1 754-707-5076', 'website': 'https://www.terrazasathyde.com/'}


2
{'amenity': 'cafe', 'name': 'Ben&jerry', 'note': 'Most Fabulous Cheesecake', 'website': 'https://www.juniorscheesecake.com/'}


3
4
5
{'amenity': 'cafe', 'cuisine': 'coffee_shop', 'name': "Polly's Gourmet Coffee"}


6
{'amenity': 'restaurant', 'name': 'Harlem Maison'}


7
{'addr:street': 'Sutphin Boulevard', 'amenity': 'cafe', 'name': 'Starbucks'}


8
{'amenity': 'cafe', 'name': 'Mod Cup'}


9
10
{'addr:housenumber': '500', 'addr:postcode': '91745', 'addr:street': 'N Atlantic Boulevard', 'amenity': 'restaurant', 'cuisine': 'Taiwan,_Chinese', 'name': 'Tasty Garden 稻香中西美食坊', 'phone': '626-872-0900'}


11
12
{'addr:housenumber': '5315', 'addr:postcode': '96761', 'addr:street': 'Lower Honoapi‘ilani Road'

In the above output I have displayed only first fetched point of interest along with its details for each coordinates.It can be seen that for few coordinates there are no POI's in 500 meter range (e.g. 3,4,9,11)

In [4]:

df['attraction']=p1
df['bar']=p15
df['cafe']=p16
df['pub']=p17
df['restaurant']=p18


df

C:\Users\sdk19\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sdk19\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sdk19\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,id,latitude,longitude,attraction,bar,cafe,pub,restaurant
0,1,25.98676,-80.12252,0,0,0,0,1
1,2,40.76198,-73.98718,5,28,24,10,145
2,3,34.00697,-118.45050,0,0,0,0,0
3,4,45.18900,-93.13541,0,0,0,0,0
4,5,33.76120,-118.13839,0,0,13,2,20
5,6,40.81575,-73.94913,0,1,1,0,9
6,7,40.70423,-73.81440,0,1,1,0,2
7,8,40.73624,-74.07250,0,0,1,0,0
8,9,41.48188,-71.42375,0,0,0,0,0
9,10,34.06883,-118.13334,0,0,0,0,5


This is how the final dataframe looks like. This 5 new features can now be used in the model to improve its performance.